In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI TensorBoard integration with Vertex AI Pipelines

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tensorboard/tensorboard_vertex_ai_pipelines_integration.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Ftensorboard%2Ftensorboard_vertex_ai_pipelines_integration.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/tensorboard/tensorboard_vertex_ai_pipelines_integration.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tensorboard/tensorboard_vertex_ai_pipelines_integration.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


## Overview

### What is Vertex AI TensorBoard

Vertex AI TensorBoard is an enterprise-ready managed version of
[Open source TensorBoard](https://www.tensorflow.org/tensorboard/get_started)
(TB), which is a Google open source project for machine learning experiment
visualization.

Vertex AI TensorBoard provides various detailed visualizations, including:

*   Tracking and visualizing metrics, such as loss and accuracy over time.
*   Visualizing model computational graphs (ops and layers).
*   Viewing histograms of weights, biases, or other tensors as they change over time.
*   Projecting embeddings to a lower dimensional space.
*   Displaying image, text, and audio samples.

In addition to powerful visualizations from
TensorBoard, Vertex AI TensorBoard provides the following benefits:

*  A persistent, shareable link to your experiment's dashboard.

*  A searchable list of all experiments in a project.

*  Tight integrations with Vertex AI services for model training.

*  Enterprise-grade security, privacy, and compliance.

With Vertex AI TensorBoard, you can track, visualize, and compare
ML experiments and share them with your team.

Learn more about [Vertex AI TensorBoard](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-introduction) and [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction).

### Objective

In this tutorial, you learn how to create a training pipeline using the KFP SDK, execute the pipeline in Vertex AI Pipelines, and monitor the training process on Vertex AI TensorBoard in near real time.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Training
- Vertex AI TensorBoard
- Vertex AI Pipelines

The steps performed include:

* Setup a service account and Google Cloud Storage buckets.
* Construct a KFP pipeline with your custom training code.
* Compile and execute the KFP pipeline in Vertex AI Pipelines with Tensorboard enabled for near real time monitoring.

### Dataset

Dataset used in this tutorial is the [flower dataset](https://www.tensorflow.org/datasets/catalog/tf_flowers) provided by TensorFlow. No other datasets are required.


### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 "kfp<2" \
                                 "google-cloud-pipeline-components==1.0.20"

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project. Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### UUID

If you're in a live tutorial session, you may be using a shared test account or project.
To avoid name collisions between users on resources created, create a  Universal Unique Identifier (uuid)
for each instance session. Append the UUID to the name of the resources you create in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts, for example, datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

## Setup service account and permissions

A service account is used to create custom training job. If you don't want to use your project's Compute Engine service account, set SERVICE_ACCOUNT to another service account ID. You can create a service account by following the [documentation instructions](https://cloud.google.com/iam/docs/creating-managing-service-accounts#creating).

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = ! gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

In [ ]:
# Grant Cloud Storage permission.
! gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT \
    --role=roles/storage.admin \
    --quiet

In [ ]:
# Grant AI Platform permission.
! gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT \
    --role=roles/aiplatform.user \
    --quiet

### Import libraries 

In [ ]:
from google.cloud import aiplatform
from google_cloud_pipeline_components.v1.custom_job.utils import \
    create_custom_training_job_op_from_component
from kfp.v2 import dsl
from kfp.v2.dsl import component

### Initialize Vertex AI SDK for Python
Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

#### Vertex AI Pipelines constants

Setup up the following constants for Vertex AI Pipelines:


In [ ]:
PIPELINE_ROOT = "{}/tensorboard-pipeline-integration/pipeline_root/".format(BUCKET_URI)
BASE_OUTPUT_DIR = "{}/pipeline-output/tensorboard-pipeline-integration-{}".format(
    BUCKET_URI, UUID
)

## Create a Vertex AI Tensorboard instance


Create a TensorBoard instance to be used by the Pipeline.

In [ ]:
TENSORBOARD_NAME = "[your-tensorboard-name]"  # @param {type:"string"}

if (
    TENSORBOARD_NAME == ""
    or TENSORBOARD_NAME is None
    or TENSORBOARD_NAME == "[your-tensorboard-name]"
):
    TENSORBOARD_NAME = PROJECT_ID + "-tb-" + UUID

tensorboard = aiplatform.Tensorboard.create(
    display_name=TENSORBOARD_NAME, project=PROJECT_ID, location=LOCATION
)
TENSORBOARD_RESOURCE_NAME = tensorboard.gca_resource.name
print("TensorBoard resource name:", TENSORBOARD_RESOURCE_NAME)

## Define Python function-based pipeline trainer component
In this tutorial, you define function-based components to train the model.
The training code is wrapped as a KFP component that is run in Vertex AI Pipeline.

Your training code must be configured to write TensorBoard logs to the Cloud Storage bucket,
the location of which the Vertex AI Training service automatically makes available using
a predefined environment variable `AIP_TENSORBOARD_LOG_DIR`.

This can usually be done by providing `os.environ['AIP_TENSORBOARD_LOG_DIR']`
as the log directory to which open source TensorBoard logs are written.

For example, in TensorFlow 2.x, you can use following code to create a `tensorboard_callback`:
```
tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'],
  histogram_freq=1)
```
and add the callback to model.fit(...)
```
# previous things
model.compile(...)

tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'],
  histogram_freq=1)
  
model.fit(dataset, epochs=10, callbacks=[tensorboard_callback])
```

In [ ]:
@component(
    base_image="tensorflow/tensorflow",
    packages_to_install=["tensorflow_datasets"],
)
def trainer(tb_log_dir_env_var: str = "AIP_TENSORBOARD_LOG_DIR"):
    """Training component."""
    import logging
    import os

    import tensorflow as tf
    import tensorflow_datasets as tfds

    IMG_WIDTH = 128

    def normalize_img(image):
        """Normalizes image.

        * Resizes image to IMG_WIDTH x IMG_WIDTH pixels
        * Casts values from `uint8` to `float32`
        * Scales values from [0, 255] to [0, 1]

        Returns:
          A tensor with shape (IMG_WIDTH, IMG_WIDTH, 3). (3 color channels)
        """
        image = tf.image.resize_with_pad(image, IMG_WIDTH, IMG_WIDTH)
        return image / 255.0

    def normalize_img_and_label(image, label):
        """Normalizes image and label.

        * Performs normalize_img on image
        * Passes through label unchanged

        Returns:
          Tuple (image, label) where
          * image is a tensor with shape (IMG_WIDTH, IMG_WIDTH, 3). (3 color
            channels)
          * label is an unchanged integer [0, 4] representing flower type
        """
        return normalize_img(image), label

    if "AIP_MODEL_DIR" not in os.environ:
        raise KeyError(
            "The `AIP_MODEL_DIR` environment variable has not been"
            + "set. See https://cloud.google.com/ai-platform-unified/docs/tutorials/image-recognition-custom/training"
        )
    output_directory = os.environ["AIP_MODEL_DIR"]

    logging.info("Loading and preprocessing data ...")
    dataset = tfds.load(
        "tf_flowers:3.*.*",
        split="train",
        try_gcs=True,
        shuffle_files=True,
        as_supervised=True,
    )
    dataset = dataset.map(
        normalize_img_and_label, num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    dataset = dataset.cache()
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(128)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

    logging.info("Creating and training model ...")
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Conv2D(
                16,
                3,
                padding="same",
                activation="relu",
                input_shape=(IMG_WIDTH, IMG_WIDTH, 3),
            ),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(512, activation="relu"),
            tf.keras.layers.Dense(5),  # 5 classes
        ]
    )
    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    # Create a TensorBoard call back and write to the gcs path provided by AIP_TENSORBOARD_LOG_DIR
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=os.environ[tb_log_dir_env_var], histogram_freq=1
    )

    # Train the model with tensorboard_callback
    model.fit(dataset, epochs=14, callbacks=[tensorboard_callback])

    logging.info(f"Exporting SavedModel to: {output_directory}")
    # Add softmax layer for intepretability
    probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
    tf.saved_model.save(probability_model, output_directory)

### Define a pipeline that uses your component

Next, define a pipeline that uses the component that was built in the previous section.

The `create_custom_training_job_op_from_component` function converts a given component into a custom training job (`CustomTrainingJobOp`) in Vertex AI.

In [ ]:
@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline. Use to determine the pipeline Context.
    name="tb-pipeline-integration",
)
def pipeline():
    custom_job_op = create_custom_training_job_op_from_component(
        trainer,
        tensorboard=TENSORBOARD_RESOURCE_NAME,
        base_output_directory=BASE_OUTPUT_DIR,
        service_account=SERVICE_ACCOUNT,
    )
    custom_job_op(project=PROJECT_ID, location=LOCATION)

## Compile the pipeline

Next, compile the pipeline.

In [ ]:
from kfp.v2 import compiler  # noqa: F811

compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="tensorboard-pipeline-integration.json"
)

## Run the pipeline

Next, run the pipeline.

In [ ]:
DISPLAY_NAME = "tb-pipeline-integration_" + UUID

job = aiplatform.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="tensorboard-pipeline-integration.json",
    pipeline_root=PIPELINE_ROOT,
)

job.run()

## Check training logs

The Vertex AI TensorBoard web app provides a visualization of logs associated with a Vertex AI TensorBoard experiment. This web application offers several tools and dashboards to visualize and compare data across experiment runs. 

Learn more see [View Vertex AI TensorBoard data](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-view).


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, **if you created the individual resources in the notebook** you can delete these resouces as follows:

In [ ]:
# Delete TensorBoard instance.
! gcloud ai tensorboards delete {TENSORBOARD_RESOURCE_NAME}

# Delete Vertex AI pipeline.
job.delete()

# Delete the Custom Job using its display name.
jobs = aiplatform.CustomJob.list(filter='display_name="Trainer"')
if jobs:
    job = jobs[0]
    job.delete()

# Delete locally generated .json file
! rm tensorboard-pipeline-integration.json

# Delete GCS bucket.
delete_bucket = False  # set True for deletion
if delete_bucket:
    ! gsutil -m rm -r {BUCKET_URI}